In [ ]:
import numpy as np
import multiprocessing 
from queue import Queue, Empty
#importi za animaciju
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

brIteracija = 0
n = 0
matrix = []
velMatrice = 0
matrix_kljuc = multiprocessing.Lock()
obradjenaPolja_kljuc = multiprocessing.Lock()
promenaIteracije_kljuc = multiprocessing.Lock()
prviRed = Queue(1)
prviRed.put(1)
# drugiRed.put(1)
redniBrojIteracije = 0
brIteracija = 0
brojacObradjenihPolja = 0
nizIteracija = list()
# nizIteracija.append([])
processes = []
servis_kljuc = multiprocessing.Lock()



def servis(i,j,nova_vrednost, broj_iteracije):
    servis_kljuc.acquire()
    nizIteracija[broj_iteracije][i][j]=vrednost
    servis_kljuc.release()
  


# funkcija koju prosledjujem tredovima kad ih kreiram
# funkcija radi sa pozicijom polja (i,j)
# pokupim staru vrednost polja(i,j) iz trenutne matrice 
# to mi je pod kljucem, zato sto menjam vrednost matrix, kada prelazim na sledecu iteraciju
def proveriZaPolje(i,j):
    # print(f"usao u tred za vrednosti {i} i {j}")
    global matricaRedova
    global matrix
    global n
    global velMatrice
    global brIteracija
    global redniBrojIteracije
    global brojacObradjenihPolja
    global prviRed
    global drugiRed
    global matrix_kljuc 
    global obradjenaPolja_kljuc
    global promenaIteracije_kljuc
    
    # matrix_kljuc.acquire()
    # staraVrednostPolja = matrix[i][j]
    # matrix_kljuc.release()


    # prolazim onoliko puta koliko je korisnik uneo da zeli da ima iteracija
    # i u redove svih mojih suseda ubacim, svoju staru(za sada jos uvek trenutnu) vrednost
    # ubacujem te vrednosti preko matrice redova koja mi je "mapirana" da se poklapa sa vrednostima i velicinom matrice
    # tj jedno polje u matrixu, ima svoj red u matriciRedova na istoj poziciji
    for k in range(brIteracija):
        while(prviRed.qsize()==0):
            continue
        print(f"nit je u novoj iteraciji {k} i redniBrojIteracije je {redniBrojIteracije} ")
        matrix_kljuc.acquire()
        staraVrednostPolja = matrix[i][j]
        matrix_kljuc.release()
        for i1 in range(i-1, i+2):
            for j1 in range(j-1, j+2):
                if(i1==i and j1==j):
                    continue;
                matricaRedova[i1%n][j1%n].put(staraVrednostPolja)

        brojZivihSuseda = 0

       
        # uzimam vrednosti iz svoga reda i pozivam task_done nad samim sobom (jer ce me susedi cekati sa join)
        # prebrojevam koliko imam zivih suseda, preko vrednosti koje sam izvukla iz reda
        for p in range(8):
            try:
                vrednostSuseda = matricaRedova[i][j].get()
                # print("uzeto iz reda")
                if vrednostSuseda == 1:
                    brojZivihSuseda+=1
                # matricaRedova[i][j].task_done()
            except Empty:
                break
        novo_stanje = 0
        # proveravam da li u sledecoj iteraciji trenutno polje treba da bude zivo ili mrtvo
        if(brojZivihSuseda<2):
            novo_stanje=0
        elif(brojZivihSuseda>3):
            novo_stanje=0
        elif(staraVrednostPolja == 1 and (brojZivihSuseda==2 or brojZivihSuseda==3)):
            novo_stanje=1
        elif(staraVrednostPolja == 0 and brojZivihSuseda==3):
            novo_stanje=1
        
        p = multiprocessing.Process(target=servis, args=(i,j,novo_stanje,redniBrojIteracije))

        obradjenaPolja_kljuc.acquire()
        brojacObradjenihPolja +=1
        mojBrojacObradjenihPolja = brojacObradjenihPolja
        obradjenaPolja_kljuc.release()

        print(f"k nam je {k} i mojBrojacObradjenihPolja je {mojBrojacObradjenihPolja}")

        if(mojBrojacObradjenihPolja==velMatrice):
            while(drugiRed.qsize() < velMatrice-1):
                continue;
            matrix = np.copy(nizIteracija[redniBrojIteracije])
            promenaIteracije_kljuc.acquire()
            redniBrojIteracije +=1
            promenaIteracije_kljuc.release()
            obradjenaPolja_kljuc.acquire()
            brojacObradjenihPolja = 0
            obradjenaPolja_kljuc.release()
            prviRed.get()
            prviRed.task_done()
            print("task done")
            # print(f"k je {k}, a redniBrojIteracije je {redniBrojIteracije}")
            drugiRed.join()
            print("drugi red join")
            prviRed.put(1)
            print(f"prvi red put {prviRed.qsize()}")
        else:
            # print("u elsu")
            drugiRed.put(1)
            print(f"dodato u drugi red za iteraciju {k} i velicina prvog reda je {prviRed.qsize()}")
            prviRed.join()
            print("prvi red join")
            drugiRed.get()
            drugiRed.task_done()
        


            
            
        # matricaRedova[i][j].join()

        

# samo kreira tredove za svako polje, pokrece i joinuje
def pokreniProcese():
    # threads = [threading.Thread(target=proveriZaPolje) for i in range(n*n)]
    global n
    global threads
    for i in range(n):
        for j in range(n):
            process = multiprocessing.Process(target=proveriZaPolje,args=(i,j))
            processes.append(process)
    test=0
    for process in processes:
        process.start()
        test+=1
        # print(test)
    # print("svi tredovi su pokrenuti")
    # print(nizIteracija)
    for process in processes:
        process.join()
        # test+=1
        # print(test)
    print(nizIteracija)




# da ne bih svaki put unosila matricu za testiranje
# n=5
#brIteracija = 10
# matrix = np.array([[0, 0, 0, 0, 0],
#                 [0, 0, 1, 0, 0],
#                 [0, 1, 1, 1, 0],
#                 [0, 0, 1, 0, 0],
#                 [0, 0, 0, 0, 0]])

n = int(input("Unesite velicinu za kvadratnu matricu sa kojom zelite igru: "))
brIteracija = int(input("Unesite broj iteracija koliko: "))

velMatrice = int(n*n)

drugiRed = Queue(velMatrice)

# za testiranje, da ne bih svaki put unosila svoju matricu
matrix = (np.array(np.random.rand(velMatrice)).reshape(n, n) > 0.5).astype(np.int8)
print(matrix)

# print("Moguce vrednosti za polja su 0(ukoliko je celija mrtva) i 1(ukoliko je celija ziva)" )
# print(f"Unesi vrednosti polja odvojenih razmakom(space-om), potrebno je da uneseti {velMatrice} brojeva: ")
# 
# # User input of entries in a 
# # single line separated by space
# entries = list(map(int, input().split()))
# 
# # For printing the matrix
# matrix = np.array(entries).reshape(n,n)

# postavljam prazne nizove u niz iteracija pa cu posle da ih menjam i ubacujem prave matrice u njega
# morala sam ovako jer me pajton zezao kad sam probala da apendujem nesto, ako vec nisam imala "vec prazno rezervisano mesto za to"
for i in range(brIteracija):
    pom = []
    matrica = np.array([[0 for j in range(n)] for x in range(n)])
    pom.append(matrica)
    nizIteracija.append(pom)
    nizIteracija[i] = np.copy(matrix)

# zatim cu dodati prvu matricu u niz

# nizIteracija[0] = np.copy(matrix)

print(f"pre tredova {nizIteracija}")

# kreiram matricu redova
matricaRedova = np.array([[Queue(8) for j in range(n)] for i in range(n)])


# pozivam za kreiranje i pokretanje tredova
pokreniProcese()



def animate(steps):
    ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
    prikazuje razvoj sistema'''
  
    def init():
        im.set_data(steps[0])
        return [im]
  
  
    def animate(i):
        im.set_data(steps[i])
        return [im]

    im = plt.matshow(steps[0], interpolation='None', animated=True);
  
    anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(steps), interval=500, blit=True, repeat=False);
    return anim


anim = animate(np.array(nizIteracija));
HTML(anim.to_html5_video())

Unesite velicinu za kvadratnu matricu sa kojom zelite igru: 5
Unesite broj iteracija koliko: 10
[[0 0 1 0 1]
 [0 0 0 1 0]
 [1 1 0 1 1]
 [1 0 1 1 0]
 [1 0 0 0 1]]
pre tredova [array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]], dtype=int8), array([[0, 0, 1, 0, 1

Process Process-40:
Process Process-46:
Process Process-47:
Process Process-32:
Process Process-48:
Process Process-33:
Process Process-28:
Process Process-50:
Process Process-35:
Process Process-38:
Process Process-49:
Process Process-30:
Process Process-36:
Process Process-31:
Process Process-39:
Process Process-41:
Process Process-37:
Process Process-42:
Process Process-43:
Process Process-34:
Process Process-26:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-27:
Process Process-29:
Process Process-45:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback 

KeyboardInterrupt: ignored

  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "<ipython-input-5-4dda3b27ec9e>", line 84, in proveriZaPolje
    vrednostSuseda = matricaRedova[i][j].get()
  File "<ipython-input-5-4dda3b27ec9e>", line 84, in proveriZaPolje
    vrednostSuseda = matricaRedova[i][j].get()
  File "<ipython-input-5-4dda3b27ec9e>", line 84, in proveriZaPolje
    vrednostSuseda = matricaRedova[i][j].get()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "/usr/lib/python3.7/queue.py", line 170, in get
    self.not_empty.wait()
  File "<ipython-input-5-4dda3b27ec9e>", line 84, in proveriZaPolje
    vrednostSuseda =